In [22]:
import os, json, time, requests
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from datetime import datetime

# -------------------------------
# Settings laden
# -------------------------------
with open("settings.json", "r") as f:
    settings = json.load(f)

allg = settings['allgemein_settings']
online = settings['online']
offline = settings['offline']
balance_settings = settings['balance']
print(f"all: {settings}")

print(f"allg: {allg}")
print(f"online: {online}")
print(f"offline: {offline}")
print(f"balance_settings: {balance_settings}")

all: {'allgemein_settings': {'model_folder': 'models', 'use_model_file': 'Heusc_v0.3_20250913_024701.keras', 'sequence_length': 60, 'train_epochs': 10000, 'batch_size': 32, 'validation_split': 0.2, 'early_stopping_patience': 5, 'feature_scaling': True, 'use_dropout': True, 'dropout_rate': 0.2, 'loss_function': 'binary_crossentropy', 'optimizer': 'adam', 'debug_mode': True}, 'offline': {'enabled': True, 'csv_folder': 'csv/binance', 'csv_file': 'BTCUSDT-1m-1y-binance-2025-09-12_22-40-01.csv'}, 'online': {'symbols': ['BTC', 'ETH', 'SOL'], 'currencies': ['USDT', 'USDT', 'USDT'], 'interval': '1m', 'poll_seconds': 10, 'max_live_train_minutes': 60}, 'balance': {'use_simulation': True, 'initial_balance': 1000, 'save_on_profit_percent': 50, 'game_over_threshold': 0, 'balance_reward_factor': 1.0, 'retrain_from_best': True}, 'training_options': {'use_weighting': True, 'punish_on_wrong': True, 'predict_confidence': True, 'log_transactions': True}}
allg: {'model_folder': 'models', 'use_model_file':

In [7]:
# -------------------------------
# Modell erstellen oder laden
# -------------------------------
def create_model(input_dim=7):
    model = Sequential([
        Dense(64, activation="relu", input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(32, activation="relu"),
        Dropout(0.2),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=Adam(0.001), loss="binary_crossentropy", metrics=["accuracy"])
    return model

model_path = os.path.join(allg['model_folder'], allg['use_model_file'])
if os.path.exists(model_path):
    model = load_model(model_path)
    print(f"Model geladen: {model_path}")
else:
    model = create_model()
    print("Neues Modell erstellt.")

# Scaler (einmal fitten im Offline Mode)
scaler = StandardScaler()
print(scaler)

Model geladen: models\Heusc_v0.3_20250913_024701.keras
StandardScaler()


In [8]:
# -------------------------------
# Offline Training (CSV)
# -------------------------------
def offline_train(csv_file):
    df = pd.read_csv(csv_file)

    # Features + Label
    X = df[["open","high","low","close","prev_close","current_close","volume"]].values
    y = (df["color"] == "green").astype(int).values

    # Scaling
    X = scaler.fit_transform(X)

    # Train
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Speichern
    model.save(model_path)
    print(f"Offline Training abgeschlossen → {model_path}")

In [9]:
# -------------------------------
# Online Mode (API)
# -------------------------------
def online_train():
    balance = balance_settings.get("initial_balance", 1000)
    position = 0
    trade_log = []
    end_time = pd.Timestamp.now() + pd.Timedelta(minutes=online['max_live_train_minutes'])

    while pd.Timestamp.now() < end_time and balance > balance_settings.get("game_over_threshold", 0):
        for sym, cur in zip(online['symbols'], online['currencies']):
            url = f"http://127.0.0.1:5000/api/train_mode?symbols={sym}{cur}&source=binance&interval={online['interval']}&period=3d"
            try:
                resp = requests.get(url)
                data = resp.json()
            except Exception as e:
                print(f"[Fehler API] {e}")
                continue

            candles = data.get(f"{sym}{cur}", {}).get("history", []) + [data.get(f"{sym}{cur}", {}).get("live")]
            for c in candles:
                if c is None: continue

                # Input Features
                X = np.array([[c["open"], c["high"], c["low"], c["close"],
                               c["prev_close"], c["current_close"], c["volume"]]])
                X = scaler.transform(X)

                # Prediction
                y_pred_prob = model.predict(X, verbose=0).flatten()[0]
                pred_label = "green" if y_pred_prob > 0.5 else "red"
                true_label = c["color"]

                profit = 0
                if pred_label == "green" and position == 0:
                    position = c["close"]
                elif pred_label == "red" and position != 0:
                    profit = (c["close"] - position) * balance_settings.get("balance_reward_factor", 1.0)
                    balance += profit
                    position = 0

                # Feedback → Retraining (eine Candle nachlernen)
                y_true = np.array([1 if true_label == "green" else 0])
                model.fit(X, y_true, epochs=1, verbose=0)

                trade_log.append({
                    "timestamp": c["timestamp"],
                    "symbol": f"{sym}{cur}",
                    "predicted": pred_label,
                    "true": true_label,
                    "conf": float(y_pred_prob),
                    "profit": profit,
                    "balance": balance
                })

        time.sleep(online["poll_seconds"])

    # Speichern
    model_name = f"{allg['use_model_file'].replace('.keras','')}_online_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras"
    model.save(os.path.join(allg['model_folder'], model_name))
    with open(model_name.replace(".keras",".json"), "w") as f:
        json.dump({"balance": balance, "trades": trade_log}, f, indent=4)

    print(f"Endbalance: {balance:.2f} | Log gespeichert.")

In [ ]:
# -------------------------------
# Aufruf
# -------------------------------
if offline["enabled"] == True:
    offline_train(settings["offline"]["csv_file"])
elif offline["enabled"] == False:
    online_train()

KeyError: 'mode'